In [1]:
pip install pandas

In [2]:
pip install ampligraph

Note: you may need to restart the kernel to use updated packages.


## IMPORT

In [4]:
import pymssql
import pandas as pd

from dateutil.relativedelta import relativedelta

import os
from os import listdir
from os.path import isfile, isdir, join

import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

import ampligraph
from ampligraph.datasets import load_from_csv
from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.latent_features import ComplEx, HolE, ConvE, TransE
from sklearn.model_selection import train_test_split
#ampligraph.__version__

C:\Users\Notebook\miniconda3\envs\depa\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Notebook\miniconda3\envs\depa\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Notebook\miniconda3\envs\depa\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Notebook\miniconda3\envs\depa\lib\site-packages\tensorf

## PROCESS

In [5]:
test = pd.read_csv('./data/SPO_cancer.csv', header = 0)
test

,subject,predicate,object
0,GSM652761,has,current_alco
1,GSM652762,has,current_alco
2,GSM652763,has,current_alco
3,GSM652764,has,never_alco
4,GSM652765,has,current_alco
...,...,...,...
2435,P484,has,no lung cancer_dise
2436,P485,has,no lung cancer_dise
2437,P486,has,lung cancer_dise
2438,P487,has,no lung cancer_dise


In [28]:
#X = load_from_csv('','./data/SPO_cancer.csv', sep=',')
#X

array([['subject', 'predicate', 'object'],
       ['GSM652761', 'has', 'current_alco'],
       ['GSM652762', 'has', 'current_alco'],
       ...,
       ['P486', 'has', 'lung cancer_dise'],
       ['P487', 'has', 'no lung cancer_dise'],
       ['P488', 'has', 'no lung cancer_dise']], dtype=object)

In [30]:
X = pd.read_csv('./data/SPO_cancer.csv')
#for col in X:
#    X[col] = X[col].astype(str)
    #X = X.to_numpy()
    #X
#    X

In [31]:
X[X['subject'].apply(lambda x : True if type(x)==float else False)]

,subject,predicate,object


In [32]:
X[X['predicate'].apply(lambda x : True if type(x)==float else False)]

,subject,predicate,object


In [33]:
X[X['object'].apply(lambda x : True if type(x)==float else False)]

,subject,predicate,object


In [34]:
X = X.to_numpy()
X

array([['GSM652761', 'has', 'current_alco'],
       ['GSM652762', 'has', 'current_alco'],
       ['GSM652763', 'has', 'current_alco'],
       ...,
       ['P486', 'has', 'lung cancer_dise'],
       ['P487', 'has', 'no lung cancer_dise'],
       ['P488', 'has', 'no lung cancer_dise']], dtype=object)

In [35]:
X_train, X_test = train_test_split(X)

In [37]:
X_test

array([['P288', 'has', '54_age'],
       ['Jack Lemmon', 'has', '56_age'],
       ['Rex Harrison', 'has', 'current_alco'],
       ...,
       ['P223', 'has', '46_age'],
       ['P240', 'has', 'cardio_dise'],
       ['P453', 'has', 'male_gender']], dtype=object)

### unique PREDICATE

In [38]:
relations = np.unique(X[1:, 1])
relations

array(['has'], dtype=object)

### train test partition

In [39]:
print('Train set size: ', X_train.shape)
print('Test set size: ', X_test.shape)

Train set size:  (1830, 3)
Test set size:  (610, 3)


### define model var

In [40]:
model = ComplEx(batches_count=100
                ,seed=0,epochs=100
                ,k=100
                ,eta=5
                ,optimizer='adam'
                ,optimizer_params={'lr':1e-3}
                ,loss='multiclass_nll'
                ,regularizer='LP'
                ,regularizer_params={'p':3, 'lambda':1e-5}
                ,verbose=True)

In [41]:
import tensorflow as tf
tf.logging.set_verbosity(tf. logging.ERROR)

### fit model

In [42]:
model.fit(X_train, early_stopping = False)

Average ComplEx Loss:   0.373448: 100%|███████████████████████████████████████████| 100/100 [00:21<00:00,  4.68epoch/s]


### check

In [50]:
from ampligraph.latent_features import save_model, restore_model
save_model(model, 'data/best_model.pkl')
#del model
model = restore_model('data/best_model.pkl')

if model.is_fitted:
    print('The model is fit!')
else:
    print('The model is not fit! Did you skip a step?')

The model is fit!


### create model folder

In [52]:
from ampligraph.utils import create_tensorboard_visualizations
create_tensorboard_visualizations(model, os.path.join('data', 'ComplEX_06'))

In [53]:
len(model.ent_to_idx)

561

In [55]:
%%bash
cd data/ComplEX_06/
tensorboard --logdir=./visualizations

-bash: line 2: tensorboard: command not found


CalledProcessError: Command 'b'cd data/ComplEX_06/\ntensorboard --logdir=./visualizations\n'' returned non-zero exit status 127.